<a href="https://colab.research.google.com/github/dbstj1231/2023_AI_Academy_ASR/blob/main/7_nemo_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nemo Installation

In [ ]:
!pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]

# Pre-trained model

Model: Conformer-large (https://www.isca-speech.org/archive/interspeech_2020/gulati20_interspeech.html)
https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_en_conformer_ctc_large


Dataset: LibriSpeech (https://openslr.org/12/)

Tokenizer: Byte-pair Encoding by Sentencepiece (https://github.com/google/sentencepiece)



## Model load

In [2]:
import nemo.collections.asr as nemo_asr

In [ ]:
model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large_ls")

## Tokenizer and Vocabulary

### Tokenizer



In [ ]:
tokenizer =


### output units

vocab: 1+127+1 (<unk> + tokens + blank)


In [ ]:
vocab =

In [7]:
with open('tokens.txt', 'w') as f:
  for k, v in enumerate(vocab):
    f.write(str(v) + ' '+str(k)+ '\n')

### words

'word.raw' file: All words in language model(librispeech dataset)


In [8]:
with open('word.raw', 'r') as f:
  wlist = f.read().splitlines()

In [19]:
len(wlist)

797682

In [10]:
with open('words.txt', 'w') as f:
  for k, v, in enumerate(wlist):
    f.write(str(v)+' '+str(k)+'\n')

## Lexicon

### wordpiece

In [11]:
pieces = []
for i in wlist:
  pieces.append(tokenizer.encode_as_pieces(i))

In [20]:
len(pieces)

797682

### Lexicon format

lexicon: word token pair (except for special symbols)

In [13]:
lexicon = list(zip(wlist, pieces))

In [14]:
with open('lexicon.txt', "w", encoding="utf-8") as f:
  for word, tokens in lexicon[1:-1]: # special symbol removal
    f.write(f"{word} {' '.join(tokens)}\n")

In [15]:
len(lexicon)

797682

# Test set logits

## Test set load

Librispeech test-clean set

HuggingFace URL: https://huggingface.co/datasets/kresnik/librispeech_asr_test



In [ ]:
!pip install datasets
from datasets import load_dataset
ds = load_dataset("kresnik/librispeech_asr_test", "clean")
test_ds = ds['test']
fl = test_ds['file']

## logit extraction

extracting log-probabilities using Nemo model and huggingface dataset

In [17]:
r = asr_model.transcribe(fl, logprobs=True)

Transcribing:   0%|          | 0/655 [00:00<?, ?it/s]

In [ ]:
r

## save logits

save logits as pt file

In [21]:
import torch

out_list = []
for i in r:
  out_list.append(torch.tensor(i))

In [ ]:
out_list

In [27]:
torch.save(out_list, 'logits.pt')

In [ ]:
out_list[0].shape

## save reference

save reference text

In [29]:
with open('ref.txt', 'w') as f:
  for i in test_ds['text']:
    f.write(i+'\n')